In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

def load_all_data(measurement_path: str) -> pd.DataFrame:
    """
    Carrega, combina e pré-processa os dados de medição de todos os arquivos CSV.
    """
    all_data = []
    
    files_map = {
        'PyLinkedList_results.csv': 'PyLinkedList',
        'PyAVLTree_results.csv': 'PyAVLTree',
        'PyRBTree_results.csv': 'PyRBTree',
        'pyskiplist_results.csv': 'PySkipList',
    }

    print(f"📊 Carregando dados do diretório: {measurement_path}")
    for file, structure_name in files_map.items():
        filepath = os.path.join(measurement_path, file)
        try:
            df = pd.read_csv(filepath)
            df['structure'] = structure_name
            all_data.append(df)
            print(f"  -> Arquivo '{file}' carregado com sucesso.")
        except FileNotFoundError:
            print(f"  -> ⚠️ AVISO: Arquivo '{file}' não encontrado em '{filepath}'. Ele será ignorado.")
            
    if not all_data:
        print("❌ ERRO: Nenhum arquivo de medição foi encontrado. Encerrando.")
        return pd.DataFrame()

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df['structure'] = pd.Categorical(combined_df['structure'], 
                                              categories=['PyLinkedList', 'PyAVLTree', 'PyRBTree', 'PySkipList'], 
                                              ordered=True)
    print("✔️  Dados combinados e prontos para plotagem.\n")
    return combined_df

def plot_comparison(data: pd.DataFrame, operation: str, sample_type: str, plot_path: str, use_log_scale: bool = False):
    """
    Gera e salva um gráfico comparativo para uma operação e tipo de amostra específicos.
    """
    # --- ALTERAÇÃO 1: Nomes de arquivo e títulos dinâmicos ---
    scale_desc = "Escala Log" if use_log_scale else "Escala Linear"
    scale_suffix = "_log_scale" if use_log_scale else "_linear_scale"
    
    title = f'Comparativo: {operation.capitalize()} ({sample_type.capitalize()}) - {scale_desc}'
    # Adicionamos o sufixo ao nome do arquivo para não sobrescrever
    filename = f'plot_{operation}_{sample_type}{scale_suffix}.png'
    filepath = os.path.join(plot_path, filename)

    print(f"🎨 Gerando gráfico: {title}")

    plot_data = data[(data['operation'] == operation) & (data['sample_type'] == sample_type)]

    if plot_data.empty:
        print(f"   -> ⚠️ Nenhum dado encontrado para '{operation}' com amostras '{sample_type}'. Gráfico ignorado.")
        return

    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(12, 8))
    
    ax = sns.lineplot(
        data=plot_data,
        x='size',
        y='time_ms',
        hue='structure',
        style='structure',
        markers=True,
        dashes=False,
        linewidth=2.5
    )

    ax.set_title(title, fontsize=18, pad=20)
    ax.set_xlabel('Tamanho da Amostra (N)', fontsize=14)
    ax.set_ylabel(f'Tempo de Execução (ms) - {scale_desc}', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

    if use_log_scale:
        ax.set_yscale('log')

    plt.legend(title='Estrutura', fontsize=12)
    plt.tight_layout()
    
    plt.savefig(filepath, dpi=300)
    plt.close()
    print(f"   -> ✅ Gráfico salvo em '{filepath}'")


def main():
    """
    Função principal que orquestra o carregamento dos dados e a geração dos gráficos.
    """
    current_dir = os.getcwd()
    project_root = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))
    
    MEASUREMENT_PATH = os.path.join(project_root, "measurements")
    PLOT_PATH = os.path.join(project_root, "plots")
    
    os.makedirs(PLOT_PATH, exist_ok=True)
    
    full_data = load_all_data(MEASUREMENT_PATH)
    
    if full_data.empty:
        return

    # --- ALTERAÇÃO 2: Adicionados 4 novos gráficos com escala linear ---
    plots_to_generate = [
        # Operação,  Tipo de Amostra, Usar Escala Log?
        ('insertion', 'sequential',   False), # Linear
        ('insertion', 'random',       False), # Linear
        ('search',    'sequential',   True),  # Log
        ('search',    'random',       True),  # Log
        ('deletion',  'sequential',   True),  # Log
        ('deletion',  'random',       True),  # Log
        # --- NOVOS GRÁFICOS COM ESCALA LINEAR ---
        ('search',    'sequential',   False), # Linear
        ('search',    'random',       False), # Linear
        ('deletion',  'sequential',   False), # Linear
        ('deletion',  'random',       False), # Linear
    ]
    
    for op, s_type, log_scale in plots_to_generate:
        plot_comparison(full_data, op, s_type, PLOT_PATH, use_log_scale=log_scale)
        
    print("\n🎉 Processo de geração de gráficos finalizado!")


if __name__ == "__main__":
    main()

📊 Carregando dados do diretório: c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\measurements
  -> Arquivo 'PyLinkedList_results.csv' carregado com sucesso.
  -> Arquivo 'PyAVLTree_results.csv' carregado com sucesso.
  -> Arquivo 'PyRBTree_results.csv' carregado com sucesso.
  -> Arquivo 'pyskiplist_results.csv' carregado com sucesso.
✔️  Dados combinados e prontos para plotagem.

🎨 Gerando gráfico: Comparativo: Insertion (Sequential) - Escala Linear
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_insertion_sequential_linear_scale.png'
🎨 Gerando gráfico: Comparativo: Insertion (Random) - Escala Linear
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_insertion_random_linear_scale.png'
🎨 Gerando gráfico: Comparativo: Search (Sequential) - Escala Log
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_search_sequential_log_scale.png'
🎨 Gerando gráfico: Comparativo: Search 

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

def load_all_data(measurement_path: str) -> pd.DataFrame:
    """
    Carrega, combina e pré-processa os dados de medição de todos os arquivos CSV.
    """
    all_data = []
    
    files_map = {
        'linkedlist_results.csv': 'LinkedList',
        'avltree_results.csv': 'AVLTree',
        'rbtree_results.csv': 'RBTree',
        'skiplist_results.csv': 'SkipList',
    }

    print(f"📊 Carregando dados do diretório: {measurement_path}")
    for file, structure_name in files_map.items():
        filepath = os.path.join(measurement_path, file)
        try:
            df = pd.read_csv(filepath)
            df['structure'] = structure_name
            all_data.append(df)
            print(f"  -> Arquivo '{file}' carregado com sucesso.")
        except FileNotFoundError:
            print(f"  -> ⚠️ AVISO: Arquivo '{file}' não encontrado em '{filepath}'. Ele será ignorado.")
            
    if not all_data:
        print("❌ ERRO: Nenhum arquivo de medição foi encontrado. Encerrando.")
        return pd.DataFrame()

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df['structure'] = pd.Categorical(combined_df['structure'], 
                                              categories=['LinkedList', 'AVLTree', 'RBTree', 'SkipList'], 
                                              ordered=True)
    print("✔️  Dados combinados e prontos para plotagem.\n")
    return combined_df

def plot_comparison(data: pd.DataFrame, operation: str, sample_type: str, plot_path: str, use_log_scale: bool = False):
    """
    Gera e salva um gráfico comparativo para uma operação e tipo de amostra específicos.
    """
    # --- ALTERAÇÃO 1: Nomes de arquivo e títulos dinâmicos ---
    scale_desc = "Escala Log" if use_log_scale else "Escala Linear"
    scale_suffix = "_log_scale" if use_log_scale else "_linear_scale"
    
    title = f'Comparativo: {operation.capitalize()} ({sample_type.capitalize()}) - {scale_desc}'
    # Adicionamos o sufixo ao nome do arquivo para não sobrescrever
    filename = f'plot_{operation}_{sample_type}{scale_suffix}.png'
    filepath = os.path.join(plot_path, filename)

    print(f"🎨 Gerando gráfico: {title}")

    plot_data = data[(data['operation'] == operation) & (data['sample_type'] == sample_type)]

    if plot_data.empty:
        print(f"   -> ⚠️ Nenhum dado encontrado para '{operation}' com amostras '{sample_type}'. Gráfico ignorado.")
        return

    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(12, 8))
    
    ax = sns.lineplot(
        data=plot_data,
        x='size',
        y='time_ms',
        hue='structure',
        style='structure',
        markers=True,
        dashes=False,
        linewidth=2.5
    )

    ax.set_title(title, fontsize=18, pad=20)
    ax.set_xlabel('Tamanho da Amostra (N)', fontsize=14)
    ax.set_ylabel(f'Tempo de Execução (ms) - {scale_desc}', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

    if use_log_scale:
        ax.set_yscale('log')

    plt.legend(title='Estrutura', fontsize=12)
    plt.tight_layout()
    
    plt.savefig(filepath, dpi=300)
    plt.close()
    print(f"   -> ✅ Gráfico salvo em '{filepath}'")


def main():
    """
    Função principal que orquestra o carregamento dos dados e a geração dos gráficos.
    """
    current_dir = os.getcwd()
    project_root = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))
    
    MEASUREMENT_PATH = os.path.join(project_root, "measurements")
    PLOT_PATH = os.path.join(project_root, "plots")
    
    os.makedirs(PLOT_PATH, exist_ok=True)
    
    full_data = load_all_data(MEASUREMENT_PATH)
    
    if full_data.empty:
        return

    # --- ALTERAÇÃO 2: Adicionados 4 novos gráficos com escala linear ---
    plots_to_generate = [
        # Operação,  Tipo de Amostra, Usar Escala Log?
        ('insertion', 'sequential',   False), # Linear
        ('insertion', 'random',       False), # Linear
        ('search',    'sequential',   True),  # Log
        ('search',    'random',       True),  # Log
        ('deletion',  'sequential',   True),  # Log
        ('deletion',  'random',       True),  # Log
        # --- NOVOS GRÁFICOS COM ESCALA LINEAR ---
        ('search',    'sequential',   False), # Linear
        ('search',    'random',       False), # Linear
        ('deletion',  'sequential',   False), # Linear
        ('deletion',  'random',       False), # Linear
    ]
    
    for op, s_type, log_scale in plots_to_generate:
        plot_comparison(full_data, op, s_type, PLOT_PATH, use_log_scale=log_scale)
        
    print("\n🎉 Processo de geração de gráficos finalizado!")


if __name__ == "__main__":
    main()

📊 Carregando dados do diretório: c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\measurements
  -> Arquivo 'linkedlist_results.csv' carregado com sucesso.
  -> Arquivo 'avltree_results.csv' carregado com sucesso.
  -> Arquivo 'rbtree_results.csv' carregado com sucesso.
  -> Arquivo 'skiplist_results.csv' carregado com sucesso.
✔️  Dados combinados e prontos para plotagem.

🎨 Gerando gráfico: Comparativo: Insertion (Sequential) - Escala Linear
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_insertion_sequential_linear_scale.png'
🎨 Gerando gráfico: Comparativo: Insertion (Random) - Escala Linear
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_insertion_random_linear_scale.png'
🎨 Gerando gráfico: Comparativo: Search (Sequential) - Escala Log
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_search_sequential_log_scale.png'
🎨 Gerando gráfico: Comparativo: Search (Random)

In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

def load_all_data(measurement_path: str) -> pd.DataFrame:
    """
    Carrega, combina e pré-processa os dados de medição de todos os arquivos CSV.
    """
    all_data = []
    
    files_map = {
        'linkedlist_results.csv': 'LinkedList',
        'avltree_results.csv': 'AVLTree',
        'rbtree_results.csv': 'RBTree',
        'skiplist_results.csv': 'SkipList',
        'PyLinkedList_results.csv': 'PyLinkedList',
        'PyAVLTree_results.csv': 'PyAVLTree',
        'PyRBTree_results.csv': 'PyRBTree',
        'pyskiplist_results.csv': 'PySkipList'
    }

    print(f"📊 Carregando dados do diretório: {measurement_path}")
    for file, structure_name in files_map.items():
        filepath = os.path.join(measurement_path, file)
        try:
            df = pd.read_csv(filepath)
            df['structure'] = structure_name
            all_data.append(df)
            print(f"  -> Arquivo '{file}' carregado com sucesso.")
        except FileNotFoundError:
            print(f"  -> ⚠️ AVISO: Arquivo '{file}' não encontrado em '{filepath}'. Ele será ignorado.")
            
    if not all_data:
        print("❌ ERRO: Nenhum arquivo de medição foi encontrado. Encerrando.")
        return pd.DataFrame()

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df['structure'] = pd.Categorical(combined_df['structure'], 
                                              categories=['LinkedList', 'AVLTree', 'RBTree', 'SkipList','PyLinkedList', 'PyAVLTree', 'PyRBTree', 'PySkipList'], 
                                              ordered=True)
    print("✔️  Dados combinados e prontos para plotagem.\n")
    return combined_df

def plot_comparison(data: pd.DataFrame, operation: str, sample_type: str, plot_path: str, use_log_scale: bool = False):
    """
    Gera e salva um gráfico comparativo para uma operação e tipo de amostra específicos.
    """
    # --- ALTERAÇÃO 1: Nomes de arquivo e títulos dinâmicos ---
    scale_desc = "Escala Log" if use_log_scale else "Escala Linear"
    scale_suffix = "_log_scale" if use_log_scale else "_linear_scale"
    
    title = f'Comparativo: {operation.capitalize()} ({sample_type.capitalize()}) - {scale_desc}'
    # Adicionamos o sufixo ao nome do arquivo para não sobrescrever
    filename = f'plot_{operation}_{sample_type}{scale_suffix}.png'
    filepath = os.path.join(plot_path, filename)

    print(f"🎨 Gerando gráfico: {title}")

    plot_data = data[(data['operation'] == operation) & (data['sample_type'] == sample_type)]

    if plot_data.empty:
        print(f"   -> ⚠️ Nenhum dado encontrado para '{operation}' com amostras '{sample_type}'. Gráfico ignorado.")
        return

    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(12, 8))
    
    ax = sns.lineplot(
        data=plot_data,
        x='size',
        y='time_ms',
        hue='structure',
        style='structure',
        markers=True,
        dashes=False,
        linewidth=2.5
    )

    ax.set_title(title, fontsize=18, pad=20)
    ax.set_xlabel('Tamanho da Amostra (N)', fontsize=14)
    ax.set_ylabel(f'Tempo de Execução (ms) - {scale_desc}', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

    if use_log_scale:
        ax.set_yscale('log')

    plt.legend(title='Estrutura', fontsize=12)
    plt.tight_layout()
    
    plt.savefig(filepath, dpi=300)
    plt.close()
    print(f"   -> ✅ Gráfico salvo em '{filepath}'")


def main():
    """
    Função principal que orquestra o carregamento dos dados e a geração dos gráficos.
    """
    current_dir = os.getcwd()
    project_root = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))
    
    MEASUREMENT_PATH = os.path.join(project_root, "measurements")
    PLOT_PATH = os.path.join(project_root, "plots")
    
    os.makedirs(PLOT_PATH, exist_ok=True)
    
    full_data = load_all_data(MEASUREMENT_PATH)
    
    if full_data.empty:
        return

    # --- ALTERAÇÃO 2: Adicionados 4 novos gráficos com escala linear ---
    plots_to_generate = [
        # Operação,  Tipo de Amostra, Usar Escala Log?
        ('insertion', 'sequential',   False), # Linear
        ('insertion', 'random',       False), # Linear
        ('search',    'sequential',   True),  # Log
        ('search',    'random',       True),  # Log
        ('deletion',  'sequential',   True),  # Log
        ('deletion',  'random',       True),  # Log
        # --- NOVOS GRÁFICOS COM ESCALA LINEAR ---
        ('search',    'sequential',   False), # Linear
        ('search',    'random',       False), # Linear
        ('deletion',  'sequential',   False), # Linear
        ('deletion',  'random',       False), # Linear
    ]
    
    for op, s_type, log_scale in plots_to_generate:
        plot_comparison(full_data, op, s_type, PLOT_PATH, use_log_scale=log_scale)
        
    print("\n🎉 Processo de geração de gráficos finalizado!")


if __name__ == "__main__":
    main()

📊 Carregando dados do diretório: c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\measurements
  -> Arquivo 'linkedlist_results.csv' carregado com sucesso.
  -> Arquivo 'avltree_results.csv' carregado com sucesso.
  -> Arquivo 'rbtree_results.csv' carregado com sucesso.
  -> Arquivo 'skiplist_results.csv' carregado com sucesso.
  -> Arquivo 'PyLinkedList_results.csv' carregado com sucesso.
  -> Arquivo 'PyAVLTree_results.csv' carregado com sucesso.
  -> Arquivo 'PyRBTree_results.csv' carregado com sucesso.
  -> Arquivo 'pyskiplist_results.csv' carregado com sucesso.
✔️  Dados combinados e prontos para plotagem.

🎨 Gerando gráfico: Comparativo: Insertion (Sequential) - Escala Linear
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_insertion_sequential_linear_scale.png'
🎨 Gerando gráfico: Comparativo: Insertion (Random) - Escala Linear
   -> ✅ Gráfico salvo em 'c:\Users\Pichau\Desktop\CC UFCG\EDALEDA\projeto-v2\plots\plot_insertion_random_linear_s